In [ ]:
from openai import OpenAI
OLAMA_BASE_URL = "http://localhost:11434/v1"
olama = OpenAI(base_url=OLAMA_BASE_URL, api_key='ollama')



In [ ]:
response=olama.chat.completions.create(
    model="llama3.2",
    messages=[{"role": "user", "content": "tell me a joke"}]
)


In [ ]:
print(response.choices[0].message.content)

In [ ]:
system_prompt = """
    You are a helpful assistant that can answer questions and help with tasks.
"""

user_prompt_prefix = """
    Go through this website 
"""

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_prefix + website}
    ]


In [ ]:
from bs4 import BeautifulSoup
import requests


# Standard headers to fetch a website
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}
def fetch_website_contents(url):
    """
    Return the title and contents of the website at the given url;
    truncate to 2,000 characters as a sensible limit
    """
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    title = soup.title.string if soup.title else "No title found"
    if soup.body:
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        text = soup.body.get_text(separator="\n", strip=True)
    else:
        text = ""
    return (title + "\n\n" + text)[:2_000]

In [ ]:
def summarize(url):
    website = fetch_website_contents(url)
    response = olama.chat.completions.create(
        model="llama3.2",
        messages=messages_for(website)
    )
    return response.choices[0].message.content


In [ ]:
summarize('https://www.google.com')